# Minimum example of applying the Christiano-Fitzgeral filter in Python

In [25]:
import pandas as pd
import yfinance as yf

download_data = True
ticker = 'BTC-USD'

if download_data:
    # Fetch daily data from Yahoo Finance
    data = yf.download(ticker, start='2021-01-01')
    data.to_parquet('data.parquet')  # write to disk to not download it for every run
else:
    data = pd.read_parquet('data.parquet')

print(data.shape)
x = data['Close']

[*********************100%%**********************]  1 of 1 completed

(1315, 6)


In [45]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_cf_filter(x, cycle, trend, lookback_window, min_period, max_period):
    # Create subplots
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True,
        vertical_spacing=0.05,
        row_heights=[0.6, 0.4],  # Make the second subplot smaller
        # subplot_titles=('Close Prices and Trend Component', 'Cycle Component')
    )

    # Close Prices and Trend Component
    fig.add_trace(go.Scatter(x=list(range(lookback_window)), y=x[-lookback_window:], 
                             mode='lines+markers', name='Close Prices', line=dict(color='black')), row=1, col=1)
    fig.add_trace(go.Scatter(x=list(range(lookback_window)), y=trend[-lookback_window:], 
                             mode='lines', name='Trend Component', line=dict(color='blue')), row=1, col=1)

    # Cycle Component
    fig.add_trace(go.Scatter(x=list(range(lookback_window)), y=cycle[-lookback_window:], 
                             mode='lines', name='Cycle Component', line=dict(color='red')), row=2, col=1)

    # Update layout for gridlines and overall appearance
    title = f'{ticker} close prices with Christiano-Fitzgerald Filter<br><sup>{min_period=}, {max_period=}</sup>'
    fig.update_layout(
        title=title, height=800, width=1600, showlegend=True,
        margin=dict(l=50, r=50, t=50, b=50),
        # plot_bgcolor='white'
    )
    fig.update_xaxes(showgrid=True, zeroline=True, zerolinewidth=1, zerolinecolor='grey', gridcolor='lightgrey')
    fig.update_yaxes(showgrid=True, zeroline=True, zerolinewidth=1, zerolinecolor='grey', gridcolor='lightgrey')

    fig.show()

# Example usage (assuming you have data for x, cycle, trend, and a lookback window)
# plot_cf_filter(x, cycle, trend, 'Christiano-Fitzgerald Filter', lookback_window)

In [68]:
from statsmodels.tsa.filters.cf_filter import cffilter

# Apply the Christiano-Fitzgerald filter
min_period, max_period = 4, 20
cycle, trend = cffilter(x, low=min_period, high=max_period, drift=False)  # Note that drift seems to not work well
plot_cf_filter(x, cycle, trend, 200, min_period, max_period)